In [ ]:
import gzip
import os
from collections import defaultdict
import random
import pandas as pd
from sklearn import linear_model
import dateutil.parser
from datetime import timedelta
from tqdm import tqdm
import csv
import pickle
import numpy as np
import xgboost as xgb
# !pip install implicit
# !pip install surprise
# from implicit import bpr
# from surprise import SVD, Reader, Dataset
# from surprise.model_selection import train_test_split

path = "/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/working_folder"
os.chdir(path)

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [ ]:
print(user_data[0])
#userName : string
#jobs : nested list [['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật XD', [[1, 1, 1998], [1, 1, 2013], 1], '', '']]
#currentPlace : ['Thành phố Hồ Chí Minh, Việt Nam', [[], 108230990, 1066296640, 1]]
#previousPlaces : [['tp. Nam Định, Nam Định, Việt Nam', [[], 204200000, 1061683330, 1]]]
#education : nested list
#gPlusUserId : string

{'userName': 'an lam', 'jobs': [['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật XD', [[1, 1, 1998], [1, 1, 2013], 1], '', '']], 'currentPlace': ['Thành phố Hồ Chí Minh, Việt Nam', [[], 108230990, 1066296640, 1]], 'previousPlaces': [['tp. Nam Định, Nam Định, Việt Nam', [[], 204200000, 1061683330, 1]]], 'education': [[[], [], [], [], [], 6], [['Đại học Kiến trúc tp Hồ chí minh, Việt nam', 'KS.XD', [[], [1, 1, 2013], 1], '', '']]], 'gPlusUserId': '100000010817154263736'}


In [ ]:
print(places_data[0])
#attributes of places_data
#name : string
#price : None /
#address : list(strings)
#hours : [['Monday', [['6:30 am--4:15 pm']], ['Tuesday', [['6:30 am--4:15 pm']]]] sometimes number like 1 is also there not sure why
#phone : string
#gPlusPlaceId : string
#gps: [float,float]

In [ ]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
state_list = set(state_list)

In [ ]:
req_places = []
for d in places_data:
  t = d["address"][-1].split()
  for ad in t:
    if ad in state_list:
      req_places.append(d)

In [ ]:
with open("USA_places.pkl","rb") as fp:
  req_places = pickle.load(fp)

In [ ]:
#places_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/places.clean.json.gz"))
user_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/users.clean.json.gz"))
#review_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/reviews.clean.json.gz"))

print("done")

done


In [ ]:
# review_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/reviews.clean.json.gz"))

In [ ]:
# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_1.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_2.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_3.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

In [ ]:
review_data1[2000]

{'categories': ['Rail Museum'],
 'gPlusPlaceId': '105347280311281389796',
 'gPlusUserId': '112235868676354256450',
 'rating': 5.0,
 'reviewText': None,
 'reviewTime': 'Feb 26, 2013',
 'reviewerName': 'sagarika Sindhu',
 'unixReviewTime': 1361932656}

In [ ]:
req_places[100000]

{'address': ['1406 W Expy 83', 'Weslaco, TX 78596'],
 'closed': False,
 'gPlusPlaceId': '103560782244834035321',
 'gps': [26176207.0, -98024823.0],
 'hours': [['Monday', [['8:00 am--9:00 pm']]],
  ['Tuesday', [['8:00 am--9:00 pm']]],
  ['Wednesday', [['8:00 am--9:00 pm']], 1],
  ['Thursday', [['8:00 am--9:00 pm']]],
  ['Friday', [['8:00 am--9:00 pm']]],
  ['Saturday', [['9:00 am--9:00 pm']]],
  ['Sunday', [['10:00 am--7:00 pm']]]],
 'name': 'Office Depot',
 'phone': '(956) 968-6207',
 'price': '$$'}

In [ ]:
# with open("USA_places.pkl","wb") as fp:
#   pickle.dump(req_places,fp)

# with open("USA_places.pkl","rb") as fp:
#   req_places = pickle.load(fp)

In [ ]:
place_ids = [d['gPlusPlaceId'] for d in req_places]
place_id_set = set(place_ids)

In [ ]:
review1_req = []
for d in review_data1:
  if d['gPlusPlaceId'] in place_id_set:
    review1_req.append(d)

In [ ]:
len(review1_req)

1846049

In [ ]:
# with open("USA_reviews1.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews2.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews3.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews1.pkl","rb") as fp:
#   review1_req = pickle.load(fp)

# with open("USA_reviews2.pkl","rb") as fp:
#   review2_req = pickle.load(fp)

with open("USA_reviews3.pkl","rb") as fp:
  review3_req = pickle.load(fp)

In [ ]:
#filtering user data
# user_ids = set([d['gPlusUserId'] for d in review3_req])

In [ ]:
with open("review1_uids.pkl","rb") as fp:
  user_ids1 = pickle.load(fp)
with open("review2_uids.pkl","rb") as fp:
  user_ids2 = pickle.load(fp)
with open("review3_uids.pkl","rb") as fp:
  user_ids3 = pickle.load(fp)

In [ ]:
# all_user_ids = user_ids1.union(user_ids2).union(user_ids3)
with open("All_req_uids.pkl","rb") as fp:
  all_user_ids = pickle.load(fp)

In [ ]:
req_users_info = []
for d in user_data:
  if d['gPlusUserId'] in all_user_ids:
    req_users_info.append(d)

In [ ]:
with open("User_Info_req.pkl","wb") as fp:
  pickle.dump(req_users_info,fp)